# Lab 2

Author: Justin Ventura [[jventura3@gulls.salisbury.edu]]

Date: Thursday, September 12th, 2020.

## - Description -

Vector & Geometry classes used to graphically represent polygons with matplotlib.  Then a few methods for all of these including vector operations, cyclic test, and a test to see if a given point is in a polygon.  This is where every test and representation will be done.

### Vector Class

In [21]:
from math import sqrt

class Vector:
    """
    This class creates an R2 vector, that is, a
    vector with an x and y component.
    Default (x,y) -> (1,1)
    """
    def __init__(self, x: float = 0, y: float = 1):
        self._x = x
        self._y = y

    # Magic Methods:
    def __repr__(self):
        return 'Vector({}, {})'.format(self._x, self._y)

    def __getitem__(self, position: int):
        return self._x if position is 0 else self._y

    # Operator Overloads.
    def __add__(self, other):
        return Vector(self._x + other._x, self._y + other._y)

    def __sub__(self, other):
        return Vector(self._x - other._x, self._y - other._y)

    def __eq__(self, other) -> bool:
        return (self.coords() == other.coords())

    # Class Attribute Getters.
    def get_x(self) -> float:
        """ Returns x value as float. """
        return self._x

    def get_y(self) -> float:
        """ Returns y value as float. """
        return self._y

    def coords(self) -> tuple:
        """ Returns tuple with x and y values. """
        return (self._x, self._y)

    # Vector Operations.
    def dot_prod(self, _v) -> float:
        """ Computes the dot product of self & other vector. """
        return sum(u_i * v_i for u_i, v_i in zip(self.coords(), _v.coords()))

    def sum_of_squares(self, _v) -> float:
        """ This function computes v_1 * v_1 + ... v_n * v_n """
        return(_v.dot_prod(_v))

    def magnitude(self, _v) -> float:
        """ This function computes length/magnitude of v """
        return sqrt(self.sum_of_squares(_v))

    def distance(self, _v) -> float:
        """ This function computes the distance between vectors u and v """
        return self.magnitude(self - _v)

### Vector Class Operation Tests:
- Addition/Subtraction
- Dot Product
- Vector Distance

In [26]:
# Vectors:
v1, v2, v3, v4 = Vector(1,1), Vector(4, 2), Vector(9, 4), Vector(69, 4.20)
print(f'v1: {v1} | v2: {v2} | v3: {v3} | v4: {v4}\n')

# Addition & Subtraction Tests:
print('------ ADD / SUB ------')
print('v1 + v2 = ', v1 + v2)
print('v3 - v1 = ', v3 - v1)

# Dot Product Tests:
print('\n------ DOT  PROD ------')
print('v2 . v3 = ', v2.dot_prod(v3)) # Should result in 44
print('v1 . v4 = ', v1.dot_prod(v4)) # Should result in 73.2

# Vector Distance Tests:
print('\n------ VECT DIST ------')
print('|v1 v3| = ', v1.distance(v3)) # Should result in ~08.544
print('|v3 v4| = ', v3.distance(v4)) # Should result in ~60.003


v1: Vector(1, 1) | v2: Vector(4, 2) | v3: Vector(9, 4) | v4: Vector(69, 4.2)

------ ADD / SUB ------
v1 + v2 =  Vector(5, 3)
v3 - v1 =  Vector(8, 3)

------ DOT  PROD ------
v2 . v3 =  44
v1 . v4 =  73.2

------ VECT DIST ------
|v1 v3| =  8.54400374531753
|v3 v4| =  60.000333332407415


### Geometry Class

In [18]:
# For book keeping purposes.
from typing import List, Tuple

# Pascal naming because my pylinter would not shut up.
VectList = List[Vector]
EdgeList = List[Tuple[Vector, Vector]]

class Geometry:
    """
    geometry class to make a polygon of vectors.
    """
    # Magic Methods:
    def __init__(self, vertices: VectList, edges: EdgeList):
        self._vertices = vertices
        self._edges = edges

    def __str__(self):
        return f'Vertices: {self._vertices}.\nEdges: {self._edges}'
    
    Utility Methods:

    def add_face(self, From: Vector = None, To: Vector = None) -> None:
        """
        This function takes two vectors, then creates an edge between
        the two.  No duplicate edges allowed.
        """

        # Req 1: vector 2 exists.
        assert(To is not None), 'Adding a face requires two vectors.'
        # Req 2: neither vectors connect already.
        assert((From, To) not in self._edges), 'Duplicate edges not allowed!'

        self._edges.append((From, To))

        # EXAMPLE:
        # [ (1,1) : [ (2,2) ]
        #   (2,2) : [ (1,1), (3,3) ]
        #   (3,3) : [ (2,2) ]
        # ]
        # TO ADD: (3,3) to (1,1)
        return None

    # def is_closed(self, origin: Vector = None) -> bool:
    #     """
    #     Determines if the given geometry is closed or not.  That is,
    #     if there is a path that goes from start, and returns back
    #     at some point.
    #     """
    #     pass

# Setting up tests:

In [19]:
# Testing values:
my_list = [[Vector(i, i), Vector(i+1, i+1)] for i in range(10)]
my_list += [[Vector(10+i, 10-i), Vector(10+i+1, 10-i-1)] for i in range(10)]
my_list.append([Vector(20, 0), (0,0)])
# for elem in my_list:
#     print(elem)

# Setting up the test polygon:
all_vertices = [x for x, _ in my_list]
poly = Geometry(all_vertices, my_list)

print(poly)

Vertices: [Vector(0, 0), Vector(1, 1), Vector(2, 2), Vector(3, 3), Vector(4, 4), Vector(5, 5), Vector(6, 6), Vector(7, 7), Vector(8, 8), Vector(9, 9), Vector(10, 10), Vector(11, 9), Vector(12, 8), Vector(13, 7), Vector(14, 6), Vector(15, 5), Vector(16, 4), Vector(17, 3), Vector(18, 2), Vector(19, 1), Vector(20, 0)].
Edges: [[Vector(0, 0), Vector(1, 1)], [Vector(1, 1), Vector(2, 2)], [Vector(2, 2), Vector(3, 3)], [Vector(3, 3), Vector(4, 4)], [Vector(4, 4), Vector(5, 5)], [Vector(5, 5), Vector(6, 6)], [Vector(6, 6), Vector(7, 7)], [Vector(7, 7), Vector(8, 8)], [Vector(8, 8), Vector(9, 9)], [Vector(9, 9), Vector(10, 10)], [Vector(10, 10), Vector(11, 9)], [Vector(11, 9), Vector(12, 8)], [Vector(12, 8), Vector(13, 7)], [Vector(13, 7), Vector(14, 6)], [Vector(14, 6), Vector(15, 5)], [Vector(15, 5), Vector(16, 4)], [Vector(16, 4), Vector(17, 3)], [Vector(17, 3), Vector(18, 2)], [Vector(18, 2), Vector(19, 1)], [Vector(19, 1), Vector(20, 0)], [Vector(20, 0), (0, 0)]]
